In [1]:
# Librerias

import os
import re
import locale
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Parametros previos
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

In [3]:
# Cambiar a nuevo directorio
new_directory = os.getcwd()
print('✔ Cambio de directorio')

# Inputs
pedidos = pd.read_excel('Pedidos\Pedidos2.xlsx',
                   usecols='C:V', sheet_name='PEDIDOS 2023'
                   )
print('✔ Archivo importado')

# Drop de columnas
columnas_a_eliminar = ['T. DE DOC','T. COMP','ENSAMBLE','TELÉFONO',
                       'DNI/RUC','CORREO','DIRECCIÓN','ESTADO PEDIDO',
                       'OBSERVACIONES','MED DE PAGO','# OPERACIÓN',
                       'MONTO','MONEDA','T.C.','CANAL DE VENTA']

for columna in columnas_a_eliminar:
    try:
        pedidos = pedidos.drop(columns=[columna])
    except KeyError:
        pass


print('✔ Columnas seleccionadas')

# Columna de fecha
column_type = pedidos['FECHA'].dtype
if column_type == 'datetime64[ns]':
    print("✔ La columna 'fecha' es de tipo datetime.")
else:
    print("X La columna 'fecha' no es de tipo datetime.")

pedidos.head()

✔ Cambio de directorio
✔ Archivo importado
✔ Columnas seleccionadas
✔ La columna 'fecha' es de tipo datetime.


C:\Users\roger\AppData\Local\Temp\ipykernel_10380\3890088881.py:6: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  pedidos = pd.read_excel('Pedidos\Pedidos2.xlsx',


,FECHA,NOMBRE Y APELLIDO,# COMPROBANTE,PRODUCTOS,ASESOR
0,2023-04-01,Example_name9587988,B01351204,TARJ. VIDEO GIGABYTE GEFORCE RTX 3070 8GB GDDR...,ALEX
1,2023-04-01,Example_name8076580,B05455712,PROC. INTEL CORE I5 11400F (BX8070811400F) 2.6...,ALEX
2,2023-04-01,Example_name1684833,B06149738,PROC. INTEL CORE I5 11400F (BX8070811400F) 2.6...,ALEX
3,2023-04-01,Example_name6613799,B08477159,"MONITOR CURVO MSI 24"" G241VC ( OPTIX G241VC ) ...",ALEX
4,2023-04-01,Example_name2408683,B08448839,GASTOS ADMINISTRATIVOS 4.64,ALEX


In [4]:
# Intervalo de fechas

start_date_picker = widgets.DatePicker(
    description='Fecha de inicio:',
    layout={'width': '400px'},
    date_format='dd-mm-yyyy'
)

end_date_picker = widgets.DatePicker(
    description='Fecha de fin:',
    layout={'width': '400px'},
    date_format='dd-mm-yyyy'
)

display(start_date_picker, end_date_picker)


DatePicker(value=None, description='Fecha de inicio:', layout=Layout(width='400px'), step=1)

DatePicker(value=None, description='Fecha de fin:', layout=Layout(width='400px'), step=1)

In [5]:
# Set fechas
fecha_inicio = pd.to_datetime(start_date_picker.value)
fecha_fin = pd.to_datetime(end_date_picker.value)

In [6]:
## Seleccion de filas con RTX y GTX (NVIDIA) dentro de su contenido de texto

# Filtro de fechas
pedidos = pedidos[pedidos['FECHA'].between(fecha_inicio, fecha_fin)]

# Definir la lista de marcas
marcas = ['ARKTEK','GIGABYTE','MSI','ASUS','ZOTAC','PNY','GALAX']

# Crear una lista para almacenar las filas desagregadas
filas_desagregadas = []

# Iterar sobre cada fila del DataFrame 'pedidos'
for index, row in pedidos.iterrows():
    # Obtener el valor de la columna 'PRODUCTOS'
    productos = row['PRODUCTOS']
    
    # Verificar si el valor es de tipo cadena y no está vacío
    if isinstance(productos, str) and productos.strip() != "":
        # Dividir el texto en líneas utilizando '\n' como separador
        lineas = productos.split('\n')
        
        # Crear una fila desagregada por cada línea que incluya 'RTX' o 'GTX'
        for linea in lineas:
            if 'RTX' in linea or 'GTX' in linea:
                # Crear una copia de la fila actual
                fila_desagregada = row.copy()
                
                # Asignar el valor de la línea desagregada a la columna 'PRODUCTOS'
                fila_desagregada['PRODUCTOS'] = linea
                
                # Agregar la fila desagregada a la lista
                filas_desagregadas.append(fila_desagregada)
    else:
        # Si el valor no es de tipo cadena o está vacío, omitir la fila
        continue

# Crear un nuevo DataFrame con las filas desagregadas
pedidos_new = pd.DataFrame(filas_desagregadas)

#################################

# Correcciones relevantes
pedidos_new['# COMPROBANTE'] = pedidos_new['# COMPROBANTE'].str.replace('\n', '')
pedidos_new['PRODUCTOS'] = pedidos_new['PRODUCTOS'].str.rstrip()
pedidos_new['PRODUCTOS'] = pedidos_new['PRODUCTOS'].str.replace(r'[^\w\s]', '', regex=False)
pedidos_new['ASESOR'].fillna('ALEX', inplace=True)

## Identificación de unidades vendidas, precio y marca

# Expresión regular para extraer los datos
patron = r'(.+)\s+(\d+UND)\s+([\d,.]+)$'

# Crear columnas para almacenar los resultados
pedidos_new['unidades'] = ''
pedidos_new['precio'] = ''
pedidos_new['marca'] = ''

# Iterar sobre las filas del DataFrame
for i, row in pedidos_new.iterrows():
    texto = row['PRODUCTOS']
    # Buscar coincidencias en el texto
    coincidencias = re.findall(patron, texto, re.MULTILINE)
    if coincidencias:
        # Extraer los valores de la coincidencia
        nombre = re.sub(patron, r'\1', texto)
        unidades = coincidencias[0][1]
        precio = coincidencias[0][2]
        # Filtrar el nombre por la lista de marcas
        marca_encontrada = next((marca for marca in marcas if marca in nombre), 'marca_NA')
        # Almacenar los valores en las columnas correspondientes
        pedidos_new.at[i, 'PRODUCTOS'] = nombre
        pedidos_new.at[i, 'unidades'] = re.findall(r'\d+', unidades)[0]  # Extraer solo el valor numérico de las unidades
        pedidos_new.at[i, 'precio'] = float(precio.replace(',', '')) / float(re.findall(r'\d+', unidades)[0])  # Dividir el monto identificado entre las unidades
        pedidos_new.at[i, 'marca'] = marca_encontrada

# Modificacion y adicion de columnas
pedidos_new['FECHA'] = pd.to_datetime(pedidos_new['FECHA'])
pedidos_new = pedidos_new.rename(columns={'PRODUCTOS': 'Product Description',
                                          '# COMPROBANTE': 'Invoice Number',
                                          'precio': 'Price Per Unit',
                                          'unidades': 'Quantity Sold/Returned',
                                          'marca': 'Manufacturer',
                                          'ASESOR': 'SalesRepName',
                                          'FECHA': 'Sold/Invoice Date(yyyy/mm/dd)',
                                          'NOMBRE Y APELLIDO': 'Shipto Company Name'})
pedidos_new['Reporter ID']=2742186
pedidos_new['Product Number']=''
pedidos_new['Invoice Line Item Number']=''
pedidos_new['Shipto Street Address1']='LIMA'
pedidos_new['Shipto Street Address2']=''
pedidos_new['Shipto City']='LIMA'
pedidos_new['Shipto Stateprov']='LIMA'
pedidos_new['Shipto Postal Code']='LIMA'
pedidos_new['Shipto Country Code']='PE'
pedidos_new['Currency of Sale']=''

# Impresiones
print('Reporte generado:')
print("✔ Fecha de inicio:", fecha_inicio)
print("✔ Fecha de fin:", fecha_fin,'\n')
print('✔ El número de registros es: ', pedidos_new.shape)

columnas_con_nulos = pedidos_new.isna().any()
columnas_con_nulos = columnas_con_nulos[columnas_con_nulos == True]
if not columnas_con_nulos.empty:
    print("- Las siguientes columnas contienen valores nulos:")
    print(columnas_con_nulos.index)
else:
    print("✔ No hay columnas con valores nulos.")

pedidos_new.head()

Reporte generado:
✔ Fecha de inicio: 2023-04-21 00:00:00
✔ Fecha de fin: 2023-05-16 00:00:00 

✔ El número de registros es:  (35, 18)
✔ No hay columnas con valores nulos.


,Sold/Invoice Date(yyyy/mm/dd),Shipto Company Name,Invoice Number,Product Description,SalesRepName,Quantity Sold/Returned,Price Per Unit,Manufacturer,Reporter ID,Product Number,Invoice Line Item Number,Shipto Street Address1,Shipto Street Address2,Shipto City,Shipto Stateprov,Shipto Postal Code,Shipto Country Code,Currency of Sale
122,2023-04-21,Example_name3206636,B08863960,TARJ. VIDEO ZOTAC GEFORCE RTX 3060 12GB GDDR6 ...,ALEX,1,413.33,ZOTAC,2742186,,,LIMA,,LIMA,LIMA,LIMA,PE,
125,2023-04-21,Example_name4916495,B07864242,TARJ. VIDEO GIGABYTE RTX 3060 TI 8GB GDDR6X EA...,ALEX,1,586.4,GIGABYTE,2742186,,,LIMA,,LIMA,LIMA,LIMA,PE,
129,2023-04-21,Example_name3999114,B03157208,TARJ. VIDEO ZOTAC GEFORCE RTX 3060 12GB GDDR6 ...,ALEX,1,413.06,ZOTAC,2742186,,,LIMA,,LIMA,LIMA,LIMA,PE,
133,2023-04-22,Example_name6334892,B04241705,TARJ. VIDEO ZOTAC GEFORCE RTX 3060 12GB GDDR6 ...,ALEX,1,413.06,ZOTAC,2742186,,,LIMA,,LIMA,LIMA,LIMA,PE,
135,2023-04-22,Example_name6949865,B09823369,TARJETA VIDEO ASUS TUF GAMING NVIDIA RTX 4070 ...,ALEX,1,1039.73,ASUS,2742186,,,LIMA,,LIMA,LIMA,LIMA,PE,


In [7]:
# Exportar el DataFrame a un archivo Excel
column_order = ['Reporter ID',
                'Product Number',
                'Product Description',
                'Manufacturer',
                'Quantity Sold/Returned',
                'Sold/Invoice Date(yyyy/mm/dd)',
                'Invoice Number',
                'Invoice Line Item Number',
                'Price Per Unit',
                'Currency of Sale',
                'Shipto Company Name',
                'Shipto Street Address1',
                'Shipto Street Address2',
                'Shipto City',
                'Shipto Stateprov',
                'Shipto Postal Code',
                'Shipto Country Code',
                'SalesRepName']

pedidos_new.to_excel('NVD-CTW-INV-WKxx-('+
                     str(fecha_inicio.day) + ' ' +
                     str.upper(fecha_inicio.strftime('%b').rstrip('. '))
                     + ' - ' +
                     str(fecha_fin.day) + ' ' +
                     str.upper(fecha_fin.strftime('%b').rstrip('. '))+')' +
                     '.xlsx',
                     index=False, encoding='utf-8', columns=column_order)


c:\Users\roger\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
